In [191]:
import numpy as np
import pandas as pd
import numba
import random
import queue
from typing import Dict
from pprint import pprint

In [192]:
T = tuple[int, int]
step_number: int = 0

@numba.njit()
def func(matrix: np.matrix, h: int, current_step: int) -> T:
    # height, weight = matrix.shape
    global step_number
    reshaped = matrix.reshape(-1)
    f: int = matrix.size - 1
    for i in range(1, reshaped.size):
        if reshaped[0, i - 1] == i:
            f -= 1
    # print(f, h, current_step)
    step_number += 1
    return f, step_number

@numba.njit()
def condition_for_exiting(result_of_func: T, h: int):
    return True if result_of_func[0] == 0 else False



In [193]:
Tree = Dict[int, tuple[np.matrix, 'Tree']]

@numba.jit(parallel=True)
def get_tree(height: int, weight: int) -> Tree:
    l = list(range(0, height * weight))
    random.shuffle(l)
    first = np.matrix(l).reshape(height, weight)
    f = func(first, 0, -1)
    tree = (None, {-1: (first, dict())})
    q = queue.PriorityQueue()
    path = [-1]  # 0 ← 1 ↑ 2 → 3 ↓
    q.put((f, first, path))
    while not condition_for_exiting(f, len(path) - 1) and q.qsize() != 0:
        el = q.get()
        matrix: np.matrix = el[1]
        path: list = el[2]
        index = np.argwhere(matrix == 0).reshape(-1)
        prev_step = path[-1]
        prev_step_tree = tree
        for i in path:
            prev_step_tree = prev_step_tree[1][i]
        if prev_step != 0 and index[1] != 0:
            new_matrix = matrix.copy()
            new_matrix[index] = new_matrix[index[0], index[1] - 1]
            new_matrix[index[0], index[1] - 1] = 0
            new_path = path + [0]
            q.put((func(new_matrix, len(path) - 1, 0), new_matrix, new_path))
            prev_step_tree[1][0] = (new_matrix, dict())
        if prev_step != 1 and index[0] != 0:
            new_matrix = matrix.copy()
            new_matrix[index] = new_matrix[index[0] - 1, index[1]]
            new_matrix[index[0] - 1, index[1]] = 0
            new_path = path + [1]
            q.put((func(new_matrix, len(path) - 1, 1), new_matrix, new_path))
            prev_step_tree[1][1] = (new_matrix, dict())
        if prev_step != 2 and index[1] != weight - 1:
            new_matrix = matrix.copy()
            new_matrix[index] = new_matrix[index[0], index[1] + 1]
            new_matrix[index[0], index[1] + 1] = 0
            new_path = path + [2]
            q.put((func(new_matrix, len(path) - 1, 2), new_matrix, new_path))
            prev_step_tree[1][2] = (new_matrix, dict())
        if prev_step != 3 and index[0] != height - 1:
            new_matrix = matrix.copy()
            new_matrix[index] = new_matrix[index[0] + 1, index[1]]
            new_matrix[index[0] + 1, index[1]] = 0
            new_path = path + [3]
            q.put((func(new_matrix, len(path) - 1, 3), new_matrix, new_path))
            prev_step_tree[1][3] = (new_matrix, dict())
    return tree[1]

C:\Users\kolya\AppData\Local\Temp\ipykernel_3552\3549605128.py:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(parallel=True)


In [194]:
tree = get_tree(4, 4)
pprint(tree)
step_number

C:\Users\kolya\AppData\Local\Temp\ipykernel_3552\3549605128.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_tree" failed type inference due to: No implementation of function Function(<bound method Random.shuffle of <random.Random object at 0x00000280A089E460>>) found for signature:
 
 >>> shuffle(list(int64)<iv=None>)
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'shuffle_impl': File: numba\cpython\randomimpl.py: Line 1857.
    With argument(s): '(list(int64)<iv=None>)':
   Rejected as the implementation raised a specific error:
     TypeError: The argument to shuffle() should be a buffer type
  raised from C:\Users\kolya\PycharmProjects\intelligent-systems\venv\Lib\site-packages\numba\cpython\randomimpl.py:1832

During: resolving callee type: Function(<bound method Random.shuffle of <random.Random object at 0x00000280A089E460>>)
During: typing of call at C:\Users\kolya

UnsupportedError: Failed in nopython mode pipeline (step: analyzing bytecode)
[1mUse of unsupported opcode (STORE_GLOBAL) found
[1m
File "..\..\..\AppData\Local\Temp\ipykernel_3552\887309397.py", line 14:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [ ]:
tree[-1][0].size

In [213]:
m = np.matrix("1, 2, 3; 4, 0, 6; 7, 8, 5")
# func(np.matrix("1, 2; 3, 0"))
height, weight = m.shape
height, weight

(3, 3)

In [214]:
m.reshape(-1)

matrix([[1, 2, 3, 4, 0, 6, 7, 8, 5]])

In [215]:
m2 = m.copy()
m2

matrix([[1, 2, 3],
        [4, 0, 6],
        [7, 8, 5]])

In [216]:
m2[1,1] = m2[1, 0]
m2[1, 0] = 0
m2, m

(matrix([[1, 2, 3],
         [0, 4, 6],
         [7, 8, 5]]),
 matrix([[1, 2, 3],
         [4, 0, 6],
         [7, 8, 5]]))

In [217]:
index = np.argwhere(m == 0).reshape(-1)
m1 = m.copy()
m1[index] = m1[index[0], index[1] - 1]
m1[index[0], index[1] - 1] = 0
m1, m

(matrix([[1, 2, 3],
         [0, 4, 4],
         [7, 8, 5]]),
 matrix([[1, 2, 3],
         [4, 0, 6],
         [7, 8, 5]]))

In [220]:
matrix = m
pprint(matrix)
new_matrix = matrix.copy()
new_matrix[index[0], index[1]] = new_matrix[index[0], index[1] - 1]
new_matrix[index[0], index[1] - 1] = 0
pprint(new_matrix)
new_matrix = matrix.copy()
new_matrix[index[0], index[1]] = new_matrix[index[0] - 1, index[1]]
new_matrix[index[0] - 1, index[1]] = 0
pprint(new_matrix)
new_matrix = matrix.copy()
new_matrix[index[0], index[1]] = new_matrix[index[0], index[1] + 1]
new_matrix[index[0], index[1] + 1] = 0
pprint(new_matrix)
new_matrix = matrix.copy()
new_matrix[index[0], index[1]] = new_matrix[index[0] + 1, index[1]]
new_matrix[index[0] + 1, index[1]] = 0
pprint(new_matrix)

matrix([[1, 2, 3],
        [4, 0, 6],
        [7, 8, 5]])
matrix([[1, 2, 3],
        [0, 4, 6],
        [7, 8, 5]])
matrix([[1, 0, 3],
        [4, 2, 6],
        [7, 8, 5]])
matrix([[1, 2, 3],
        [4, 6, 0],
        [7, 8, 5]])
matrix([[1, 2, 3],
        [4, 8, 6],
        [7, 0, 5]])


In [224]:
def func(matrix: np.matrix, h: int, current_step: int) -> T:
    # height, weight = matrix.shape
    f: int = matrix.size - 1
    reshaped = matrix.reshape(-1)
    for i in range(1, reshaped.size):
        if reshaped[int(0), i - 1] == i:
            f -= 1
    # print(f, h, current_step)
    return f, h, current_step

def condition_for_exiting(result_of_func: T, h: int, step_number: int):
    return True if result_of_func[0] == 0 else False

condition_for_exiting(func(np.matrix("1, 2; 3, 0"), 0, 0), None, None)

True

In [230]:
m3 = np.matrix("1, 2; 3, 0")
m4 = np.matrix("1, 2; 3, 0")

In [236]:
m3.tobytes() == m4.tobytes()

True

In [232]:
hash(m3), hash(m4), hash(m3) == hash(m4)

TypeError: unhashable type: 'numpy.ndarray'

In [239]:
type(hash(m3.tobytes()))

int

In [245]:
mem: memoryview = m3.data

In [250]:
mem

memoryview